
このノートブックを実行するには、次の追加ライブラリが必要です。 Colab での実行は実験的なものであることに注意してください。問題がある場合は、Github の問題を報告してください。


In [ ]:
!pip install d2l==1.0.0-beta0



# カスタムレイヤー

ディープラーニングの成功の要因の 1 つは、さまざまなタスクに適したアーキテクチャを設計するために創造的な方法で構成できる幅広いレイヤーが利用できることです。たとえば、研究者は、画像、テキストの処理、連続データのループ、動的プログラミングの実行に特化したレイヤーを発明しました。遅かれ早かれ、深層学習フレームワークにまだ存在していないレイヤーに遭遇したり、発明したりすることになるでしょう。このような場合は、カスタム レイヤーを構築する必要があります。このセクションでは、その方法を説明します。


In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l


## (**パラメータのないレイヤー**)

まず、独自のパラメーターを持たないカスタム レイヤーを構築します。 :numref: `sec_model_construction`のモジュールの紹介を思い出していただければ、これに見覚えがあるはずです。次の`CenteredLayer`クラスは、入力から平均を単純に減算します。これを構築するには、基本層クラスから継承し、順伝播関数を実装するだけです。


In [2]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()


レイヤーにデータをフィードして、レイヤーが意図したとおりに機能することを確認してみましょう。


In [3]:
layer = CenteredLayer()
layer(torch.tensor([1.0, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])


これで、[**より複雑なモデルを構築する際に、レイヤーをコンポーネントとして組み込むことができます。** 】


In [4]:
net = nn.Sequential(nn.LazyLinear(128), CenteredLayer())


追加の健全性チェックとして、ネットワーク経由でランダム データを送信し、平均が実際に 0 であることを確認できます。浮動小数点数を扱っているため、量子化により非常に小さな非ゼロの数値が表示される可能性があります。


In [5]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(0., grad_fn=<MeanBackward0>)


## [**パラメータ付きレイヤー**]

単純なレイヤーを定義する方法がわかったので、トレーニングを通じて調整できるパラメーターを使用してレイヤーを定義することに進みます。組み込み関数を使用してパラメータを作成し、基本的なハウスキーピング機能を提供できます。特に、アクセス、初期化、共有、保存、モデル パラメーターの読み込みを制御します。この方法には、特に、カスタム レイヤーごとにカスタム シリアル化ルーチンを作成する必要がなくなるという利点があります。

次に、独自のバージョンの完全接続層を実装してみましょう。この層には 2 つのパラメーターが必要であることを思い出してください。1 つは重みを表し、もう 1 つはバイアスを表します。この実装では、ReLU アクティベーションをデフォルトとして焼き付けます。この層には 2 つの入力引数、 `in_units`と`units`が必要です。これらはそれぞれ入力と出力の数を示します。


In [6]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)


次に、 `MyLinear`クラスをインスタンス化し、そのモデル パラメーターにアクセスします。


In [7]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-1.2894e+00,  6.5869e-01, -1.3933e+00],
        [ 7.2590e-01,  7.1593e-01,  1.8115e-03],
        [-1.5900e+00,  4.1654e-01, -1.3358e+00],
        [ 2.2732e-02, -2.1329e+00,  1.8811e+00],
        [-1.0993e+00,  2.9763e-01, -1.4413e+00]], requires_grad=True)


[**カスタム レイヤーを使用して順伝播計算を直接実行できます。** 】


In [8]:
linear(torch.rand(2, 5))

tensor([[0.0000, 1.7772, 0.0000],
        [0.0000, 1.0303, 0.0000]])


(**カスタム レイヤーを使用してモデルを構築する) こともできます。**これを取得したら、組み込みの完全に接続されたレイヤーと同じように使用できます。


In [9]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])


## まとめ

基本レイヤー クラスを介してカスタム レイヤーを設計できます。これにより、ライブラリ内の既存のレイヤーとは異なる動作をする柔軟な新しいレイヤーを定義できます。カスタム レイヤーを定義すると、任意のコンテキストおよびアーキテクチャで呼び出すことができます。レイヤーにはローカル パラメーターを含めることができ、これは組み込み関数を通じて作成できます。

## 演習
1. 入力を受け取り、テンソル リダクションを計算する層を設計します。つまり、$y_k = \sum_{i, j} W_{ijk} x_i x_j$ を返します。
1. データのフーリエ係数の前半を返す層を設計します。



[ディスカッション](https://discuss.d2l.ai/t/59)
